In [7]:
import numpy as np

In [8]:
S0 = 36
sgm = 0.2
T = 1.0
K = 40
q = 0.00
N = 150
rf = 0.06
M = 150000
alpha = 0.5

In [9]:
def MLSM(S0, vol, div, r, alpha, K, T, N, M):
    np.random.seed(123)
    dt = float(T) / N
    df = np.exp(-r * dt)
    S = np.zeros((N + 1, M))
    S[0, :] = S0 * np.exp(alpha * vol * np.sqrt(T) * np.random.standard_normal(M))

    for t in range(1, N + 1):
        ran = np.random.standard_normal(int(M / 2))
        ran = np.concatenate((ran, -ran))
        S[t, :] = S[t - 1, :] * np.exp((r - div - 0.5 * vol ** 2 ) * dt + vol * ran * np.sqrt(dt))

    h = np.maximum(K - S, 0)
    V = np.zeros_like(h)
    V[-1] = h[-1]

    for t in range(N - 1, 0, -1):
        rg = np.polyfit(S[t, :], V[t + 1, :] * df, 5)
        C = np.polyval(rg, S[t, :])
        V[t, :] = np.where(h[t, :] > C, h[t, :],  V[t + 1, :] * df)
    
    rg0 = np.polyfit(S[0, :], V[1, :] * df , 5)
    V0 = np.polyval(rg0, S0)
    dV = np.polyder(rg0)
    dV2 = np.polyder(rg0, 2)
    delta = np.polyval(dV, S0)
    gamma = np.polyval(dV2, S0)

    print(f"S0: {S0}| Vol: {vol}| Expiry: {T:.4f}| Option Price = {V0:.4f}, Delta = {delta:.5f}, Gamma = {gamma:.4f}")

In [10]:
MLSM(S0, sgm, q, rf, alpha,K, T, N, M)

S0: 36| Vol: 0.2| Expiry: 1.0000| Option Price = 4.4630, Delta = -0.72069, Gamma = 0.0813
